In [1]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
# import seaborn as sns
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
# dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.distributed import Client, LocalCluster

import scipy.integrate as integ
warnings.filterwarnings("ignore")




/homes/metogra/iufarias/xroms/xroms/interp.py:18: UserWarning: xESMF is not installed, so `interpll` will not run.
  warnings.warn("xESMF is not installed, so `interpll` will not run.")
/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/pyproj/__init__.py:90: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
client = Client(n_workers=80)
# client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 80
Total threads: 160,Total memory: 503.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34994,Workers: 80
Dashboard: http://127.0.0.1:8787/status,Total threads: 160
Started: Just now,Total memory: 503.40 GiB
Comm: tcp://127.0.0.1:32789,Total threads: 2
Dashboard: http://127.0.0.1:44156/status,Memory: 6.29 GiB
Nanny: tcp://127.0.0.1:35990,


## Reading CROCO (3D)

In [4]:
# chunks={'time_counter':10, 'x_rho':50, 'y_rho':190,'s_rho':80,'s_w':80}
chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
                        'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}


path='/data/pacific/lrenault/SASI/CROCO/'

varf=xr.open_mfdataset(path + 'FULL/SASI*2012-06*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-06*.nc',data_vars='minimal',chunks=chunks)

# varf=xr.open_mfdataset(path + 'FULL/SASI*2012*.nc',data_vars='minimal',chunks=chunks)
# vart=xr.open_mfdataset(path + 'SMTH/SASI*2012*.nc',data_vars='minimal',chunks=chunks)



Converting to xroms

In [5]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

In [6]:

# plt.rcParams['text.usetex'] = True
# plt.rcParams.update({'font.size': 22})

# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['font.sans-serif'] = ['Lucida Grande']

In [7]:
timer_h=pd.DatetimeIndex(dt.ocean_time)
timer_d=pd.DatetimeIndex(dt.ocean_time.groupby("ocean_time.dayofyear").mean())

#### Spectral parameters

In [8]:
dx=0.5
nf=2
wdws='hann' #'flattop','hann'
wdw_cor=True
scl='density'

In [61]:
z_interp=np.linspace(-5,-50, 10)
fnum=240

# $N^2_r$
JAN=2.275e-06
FEB=4.178e-06
MAR=2.116e-05
APR=2.863e-05
MAY=1.790e-05
JUN=5.648e-05
AVG=2.175e-05

In [56]:
rho_full=df.rho

In [57]:
z=rho_full.z_rho0.compute()

In [58]:
%%time

N2=[]
for ixx in range(rho_full.shape[0]):
    N2.append(df.xroms.N2[:,:,120:-120,120:800][ixx,-2].compute().mean())

CPU times: user 3min 36s, sys: 1min 8s, total: 4min 45s
Wall time: 13min


In [59]:
np.mean(N2)

5.648094147306015e-05

### EKE (FULL)

In [39]:
EKE_spec_full_zt=xr.DataArray(coords=(range(df.u.shape[0]), range(z_interp.shape[0]),range(fnum)),
            dims=['ocean_time','z_rho_psi','freq_r'])

In [40]:
%%time
u_full_psi=df.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
v_full_psi=df.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]

for ix in range(df.u.shape[0]):


    v_interp_full=v_full_psi[ix].xroms.isoslice(z_interp).compute()
    u_interp_full=u_full_psi[ix].xroms.isoslice(z_interp).compute()


    Vf_spec=xrft.isotropic_power_spectrum(v_interp_full,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)
    Uf_spec=xrft.isotropic_power_spectrum(u_interp_full,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EKE_spec_full_zt[ix,:,:]=0.5*(Uf_spec.data**2 + Vf_spec.data**2)

CPU times: user 31min 53s, sys: 1h 5min 17s, total: 1h 37min 10s
Wall time: 29min 52s


In [53]:
EKE_spec_full_zt['freq_r']=Vf_spec.freq_r.data
EKE_spec_full_zt['ocean_time']=df.ocean_time.data
EKE_spec_full_zt['z_rho_psi']=z_interp

In [58]:
month='06'

In [60]:
%%time
EKE_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Eke_full_{0}.nc'.format(month))

CPU times: user 16 ms, sys: 6.97 ms, total: 23 ms
Wall time: 44.9 ms


### EKE (SMTH)

In [68]:
EKE_spec_smth_zt=xr.DataArray(coords=(range(dt.u.shape[0]), range(z_interp.shape[0]),range(fnum)),
            dims=['ocean_time','z_rho_psi','freq_r'])

In [69]:
%%time
u_smth_psi=dt.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
v_smth_psi=dt.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]

for ix in range(dt.u.shape[0]):


    v_interp_smth=v_smth_psi[ix].xroms.isoslice(z_interp).compute()
    u_interp_smth=u_smth_psi[ix].xroms.isoslice(z_interp).compute()


    Vt_spec=xrft.isotropic_power_spectrum(v_interp_smth,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)
    Ut_spec=xrft.isotropic_power_spectrum(u_interp_smth,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EKE_spec_smth_zt[ix,:,:]=0.5*(Ut_spec.data**2 + Vt_spec.data**2)

CPU times: user 31min 43s, sys: 1h 5min 36s, total: 1h 37min 20s
Wall time: 27min 47s


In [72]:
EKE_spec_smth_zt['freq_r']=Vt_spec.freq_r.data
EKE_spec_smth_zt['ocean_time']=dt.ocean_time.data
EKE_spec_smth_zt['z_rho_psi']=z_interp

In [73]:
%%time
EKE_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Eke_smth_{0}.nc'.format(month))

CPU times: user 21.2 ms, sys: 4.6 ms, total: 25.8 ms
Wall time: 47.5 ms


### EPE (FULL)

In [101]:
N2r=2.175e-05

In [96]:
EPE_spec_full_zt=xr.DataArray(coords=(range(df.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [98]:
%%time
rho_full_psi=df.rho[:,:,120:-120,120:800]


for iy in range(df.rho.shape[0]):
    rho_interp_full=rho_full_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_full=xroms.buoyancy(rho_interp_full)


    Bf_spec=xrft.isotropic_power_spectrum(b_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EPE_spec_full_zt[iy,:,:]=(0.5/N2r)*(Bf_spec.data**2)

CPU times: user 14min, sys: 32min 4s, total: 46min 5s
Wall time: 6min 29s


In [104]:
EPE_spec_full_zt['freq_r']=Bf_spec.freq_r.data
EPE_spec_full_zt['ocean_time']=df.ocean_time.data
EPE_spec_full_zt['z_rho']=z_interp

In [106]:
EPE_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_full_{0}.nc'.format(month))

### EPE(SMTH)

In [107]:
EPE_spec_smth_zt=xr.DataArray(coords=(range(dt.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [118]:
%%time
rho_smth_psi=dt.rho[:,:,120:-120,120:800]


for iy in range(dt.rho.shape[0]):
    rho_interp_smth=rho_smth_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_smth=xroms.buoyancy(rho_interp_smth)


    Bt_spec=xrft.isotropic_power_spectrum(b_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EPE_spec_smth_zt[iy,:,:]=(0.5/N2r)*(Bt_spec.data**2)

CPU times: user 15min 44s, sys: 33min 26s, total: 49min 11s
Wall time: 13min 57s


In [119]:
EPE_spec_smth_zt['freq_r']=Bt_spec.freq_r.data
EPE_spec_smth_zt['ocean_time']=dt.ocean_time.data
EPE_spec_smth_zt['z_rho']=z_interp

In [120]:
EPE_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_smth_{0}.nc'.format(month))

### w'b' (FULL)

In [127]:
wb_spec_full_zt=xr.DataArray(coords=(range(df.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [128]:
%%time
w_full_psi=df.w[:,:,120:-120,120:800]
rho_full_psi=df.rho[:,:,120:-120,120:800]

for iy in range(df.rho.shape[0]):
    rho_interp_full=rho_full_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_full=xroms.buoyancy(rho_interp_full)
    
    w_interp_full=w_full_psi[iy].xroms.isoslice(z_interp).compute()


    WBf_spec=xrft.isotropic_cross_spectrum(w_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                           b_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real

    wb_spec_full_zt[iy,:,:]=(WBf_spec.data)

CPU times: user 28min 56s, sys: 1h 7min 8s, total: 1h 36min 5s
Wall time: 19min 38s


In [130]:
wb_spec_full_zt['freq_r']=WBf_spec.freq_r.data
wb_spec_full_zt['ocean_time']=df.ocean_time.data
wb_spec_full_zt['z_rho']=z_interp

In [131]:
wb_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_full_{0}.nc'.format(month))

### w'b' (SMTH)

In [133]:
wb_spec_smth_zt=xr.DataArray(coords=(range(dt.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [ ]:
%%time
w_smth_psi=dt.w[:,:,120:-120,120:800]
rho_smth_psi=dt.rho[:,:,120:-120,120:800]

for iy in range(dt.rho.shape[0]):
    rho_interp_smth=rho_smth_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_smth=xroms.buoyancy(rho_interp_smth)
    
    w_interp_smth=w_smth_psi[iy].xroms.isoslice(z_interp).compute()


    WBt_spec=xrft.isotropic_cross_spectrum(w_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                           b_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real

    wb_spec_smth_zt[iy,:,:]=(WBt_spec.data)

In [ ]:
wb_spec_smth_zt['freq_r']=WBt_spec.freq_r.data
wb_spec_smth_zt['ocean_time']=dt.ocean_time.data
wb_spec_smth_zt['z_rho']=z_interp

In [ ]:
wb_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_smth_{0}.nc'.format(month))

In [251]:
# fig,ax=plt.subplots(2,1,figsize=(9,6))


# im1=ax[0].pcolormesh(AA.freq_r/dx,AA.z_rho_psi,AA.mean(axis=0)/dx*1e3,
#                vmin=0,vmax=6e-5,cmap='inferno')
# ax[0].set_xscale('log')
# ax[0].set_xlim(1e-2,0)
# ax[0].set_title(r'(a)',color='k',fontsize=20)
# # ax[0].set_ylabel('Depth [m]',fontsize=20,x=0.1)



# im2=ax[1].pcolormesh(BB.freq_r/dx,BB.z_rho_psi,BB.mean(axis=0)/dx*1e3,
#                vmin=0,vmax=6e-5,cmap='inferno')
# ax[1].set_xscale('log')
# ax[1].set_xlim(1e-2,0)
# ax[1].set_title(r'(b)',fontsize=20)
# # ax[1].set_ylabel('Depth [m]',fontsize=20,x=0.1)
# fig.supylabel('Depth [m]',fontsize=20)
# fig.supxlabel(r'Wavenumber [km$^{-1}$]',fontweight='bold',fontsize=20,y=-0.00001)
    

    
# fig.subplots_adjust(right=0.8,hspace=0.4)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# cbar=fig.colorbar(im, cax=cbar_ax)
# # cbar=fig.colorbar(im,cax=ax[0]
# cbar.set_label(r'Spectral Density [m$^3\,$s$^{-3}$]',labelpad=12)

# # plt.tight_layout()
# plt.savefig('/homes/metogra/iufarias/Documents/figures/2_CROCO/CROCO_zintegrated/z_int/wb_z.png')